# Code Examples – Hot Delivery Optimization Project

### Example 1: Basic Distance Minimization (Part I)
This example solves a routing problem where a single driver starts at a fixed location and must pick up and drop off orders while minimizing total travel distance.

In [ ]:

# Extracting route from the solution
edges = {(i,j): var.value() for (i,j), var in x.items() if var.value() == 1}
succ = {i: j for (i,j) in edges}

route = []
current = 0
while True:
    nxt = succ.get(current)
    if nxt is None or nxt == 0:
        break
    route.append((index_loc[current], index_loc[nxt]))
    current = nxt

for step, (frm, to) in enumerate(route, start=1):
    print(f"{step}. {frm} → {to}")


### Example 2: Add Time Constraints (Part II)
This example introduces ready times and computes the arrival time at each location to ensure the driver does not arrive before the order is ready.

In [ ]:

# Enforcing ready time for each pickup
for idx, (pick_loc, ready_time) in enumerate(zip(pickup_locations, ready_times)):
    pi = loc_index[pick_loc]
    ready_min = (ready_time - reference_time).total_seconds() / 60
    model += t[pi] >= ready_min


### Example 3: Compute Average Customer Wait Time
This block ensures the average time customers wait from when the order is ready to when it is delivered does not exceed 30 minutes.

In [ ]:

total_wait_time = pulp.lpSum([
    t[loc_index[dropoff]] - ((rt - reference_time).total_seconds() / 60)
    for dropoff, rt in zip(dropoff_locations, ready_times)
])
model += total_wait_time / len(pickup_locations) <= 30


### Example 4: Multi-Driver Round-Robin Assignment (Part III)
This snippet assigns orders to drivers in a round-robin fashion as a baseline for comparison with the optimized model.

In [ ]:

order_assignments = {d: [] for d in range(n_drivers)}
for i, idx in enumerate(orders_df.index):
    order_assignments[i % n_drivers].append(idx)
